My room bookcase
=============

# Klimpen bookcase
http://www.ikea.com/it/it/catalog/products/10353765/

![klimpen full size](./images/klimpen_1.png)
![klimpen detail](./images/klimpen_2.png)

We will build the various part of the bookcase, then we will put them together. A second approach will be shown after (PyPlasm $Q(arg)$ function will be used).
Structure:
- back: as suggested is the back side of our bookcase, it will be attached to the shelves, the side block, the top and the bottom.
- shelves
- top
- bottom
Kplimpen is offered as a 50x70x33 double shelves bookcase. For this woorkshoop we will handle a parametrized (in respec of the dimension and color) Klimpen


Note: measures are in meters

In [1]:
from larlib import *

Evaluating fenvs.py..
...fenvs.py imported in 0.009982 seconds


# Bookcase structure
We will build the various part of the bookcase, then we will put them together.
A second approach will be shown after (PyPlasm $Q(arg)$ function will be used).

Structure:
- back: as suggested is the back side of our bookcase, it will be attached to the shelves, the side block, the top and the bottom.
- shelves
- top
- base

Kplimpen is offered as a 50x70x33 double shelves bookcase. For this woorkshoop we will handle a parametrized (in respect of the dimension and color) Klimpen

## Back of the bookcase

In [2]:
def back(width, height):
    """
    back: builds the back of the bookcase
    :param back: height, width in meters
          thickness is fixed
    :return: a CUBOID. The 3d model for the back
    """ 
    return CUBOID([width,height,0.01]) #0.1 is thickness

In [12]:
b = back(0.58, 0.7)
VIEW(b)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1817bbc660> >

## Shelves

# Shelves
We are going to create one and more shelves.
firstInitSpace returns the proper offset from the base for the first shelf from the bottom. This is usefull when building the bookcase where the traslation operation is used. Traslation operation is not "relative" to a point butabsolute.

In [165]:
def shelf(width, height, length):
    """
    shelf: builds a single shelf
    :param width: width of the shelf
    :return: a CUBOID. The 3D model for a shelf
    """
    return CUBOID([width,height,length])


def shelves(nshelves, heightBack, width, height, length):
    """
    shelves builds <nshelves> shelves
    :param nshelves: number of shelves
    :param heightBack: height of the back, the area where shelves are positionable
    :param width: width of the shelf.
    :param height: height of the shelf.
    :param length: length of the shelf. (z-axis)
    :return: nshelves CUBOIDs. nshelves shelves
    """
    space = heightBack/(nshelves+1)-height
    s = STRUCT(NN(nshelves) ([shelf(width, height, length), T(2)(space)]))
    return s

def shelvesInitSpace(nshelves, heightBack, heightShelf):
    """
    :param nshelves: number of shelves 
    :param heightBack: height of the back
    :param heightShelf: height of the shelf
    :return: Long. Space from the first shelf and the base
    """
    space =  heightBack/(nshelves+1)-heightShelf
    return  space

In [167]:
ss = shelves(nshelves=2, heightBack=0.58, width=0.68, height=0.02, length=0.3)
VIEW(ss)
VIEW(T(2)(shelvesInitSpace(2, 0.58, 0.02))(ss))

![2 shelves](./images/2_shelves.png)

## Sides
The lateral components. Two parallel wooden beam

In [4]:
def side(heigth, length):
    """
    side: side of the bookcase structure
    :param heigth: height of the wooden beam
    :param length: length of the wooden beam
    :return: a CUBOID. The 3d model for a lateral wooden beam
    """
    return CUBOID([0.03, heigth, length])

In [5]:
def sides(height, length, distance):
    """
    sides: a couple of sides
    :param height: as side(height,length)
    :param length:  "
    :param distance:  "
    :return: STRUCT([CUBOID,CUBOID])
    """
    s = side(height, length)
    sides = STRUCT([s, T(1)(distance)(s)])
    return sides

In [16]:
single_side = side(0.58, 0.3)
VIEW(single_side)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1817bbc630> >

In [18]:
two_sides = sides(0.58, 0.3, 0.7)
VIEW(two_sides)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1817bbcb40> >

The lateral components of our bookcase

![sides](./images/sides.png)

## Top
The top of our bookcase. It consists of a set of CUBOIDs placed one on top of another

In [9]:
def top(widthSmall, heightSmall, lengthSmall, widthBookcase, lengthBookcase):
    """
    top: the top of the bookcase, cmade of two different layers/block
    _____________________
    [                   ]
    [      layerBig     ]
    [___________________]
      [ layerSmall    ]
      [_______________]
    
    :param widthSmall: the width of layerSmall, the same of the shelf
    :param heightSmall: the height, see above
    :param lengthSmall: the length, see above
    
    :param widthBookcase: the width of the back side of the boockase will be used to calculate the top width
    :param lengthBack: the length of the back
    :return: a STRUCT made of CUBOIDs (layerBig/small).
    """
    #layer small should have the same dimension of the shelves
    layerSmall = shelf(widthSmall, heightSmall, lengthSmall)
    
    #offset between the two layers
    offX = (widthBookcase-widthSmall)/2
    offZ = (lengthBookcase-lengthSmall)/2
    
    layerBig = CUBOID([widthBookcase, 0.03, lengthBookcase])
    
    return STRUCT([layerSmall, T([1,2,3])([-offX,heightSmall,-offZ])(layerBig)])

# Usefull variables
This value should be aproximately the ones for the standard ikea Klimpen but may be customized

In [136]:
sizes = {
'backWidth': 0.61,
'backHeight': 0.48,
'shelfWidth': 0.61,
'shelfHeight': 0.02,
'shelfLength': 0.3,
'bookcaseWidth': 0.7,
'bookcaseHeight': 0.5,
'bookcaseLength': 0.33
}

We should keep in mind that the bottom and small layer of the top is the same size as a shelf

In [13]:
t = top(shelfWidth, shelfHeight, shelfLength, shelfWidth+0.04, shelfLength+0.04)
VIEW(t)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x117682ea0> >

![top](./images/top.png)

## Base
As the top the base is made of layers, we have two different layers: 
- the one on the bottom that is the biggest of the two
- the middle one that is very thin and has a smaller perimeter than the lower one

In [14]:
def base(width, length):
    """
    :param width: width of the bookcase
    :param length: length of the bookcase
    :return: STRUCT. Two CUBOIDs representing the base
    """
    
    layerBig = CUBOID([width, 0.05, length])
    layerSmall = CUBOID([0.96*width, 0.01, 0.96*length])
    
    #offsets. layerSmall has a 0.04^2 smaller perimeter (X,Z)
    offX = width*0.02
    offZ = length*0.02
    
    b = STRUCT([layerBig, T([1, 2, 3])([offX, 0.05, offZ])(layerSmall)])
    return b

In [66]:
b = base(bookcaseWidth, bookcaseLength)
VIEW(b)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x117746de0> >

![base](./images/base.png)

# Let's build the KLIMPEN bookcase
Now we will create a model for the whole bookcase.
The ggpl_KLIMPEN function takes as reference top/base width and length.

In [162]:
def ggpl_KLIMPEN(nshelves, sizes, colorHEX):
   
    # base
    baseKlimpen = base(sizes['bookcaseWidth'], sizes['bookcaseLength'])
    
    # back
    # back is traslated according to the position of the base 
    backKlimpen = T([1, 2, 3])([0.03, 0.06, 0.01])(back(sizes['backWidth'],sizes['backHeight']))
    
    # shelves
    # shelves are traslated according to the position of the base
    shelvesOffset
    shelvesKlimpen = T(2)(shel)(shelves(nshelves, sizes['backHeight'], sizes['shelfWidth'], sizes['shelfHeight'], sizes['shelfLength'])))
    
    # Top
    #topKlimpen = T([1,2,3])([0.01, 0.06+sizes['backHeight'], ])(top(sizes['shelfWidth'], sizes['shelfHeight'], sizes['shelfLength'], sizes['bookcaseWidth'], sizes['bookcaseLength']))
    
    # sides
    # sidesOffsetX is the offset on the x-axis. 
    sidesOffsetX = (sizes['bookcaseWidth'] - sizes['backWidth'] + 0.03)/4
    sidesKlimpen = T([1, 2, 3])([sidesOffsetX, 0.06, 0.01])(sides(sizes['backHeight'], sizes['shelfLength']+0.01, sizes['backWidth']))
    
    
    klimpen = HEX(colorHEX)(STRUCT([baseKlimpen, backKlimpen, sidesKlimpen, shelvesKlimpen]))
    
    return klimpen
    

In [163]:
myKlimpen = ggpl_KLIMPEN(2, sizes, "#ffdb87")
VIEW(myKlimpen)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1177c7a20> >